In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('practice-347601-6e0b822fd842.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#
date = 0 #当日は0、前日は1・・・
#date = 1 #当日は0、前日は1・・・
date2 = "08/08"
pmc = 23012008 # 店舗ID
SPREADSHEET_KEY = '1B75CIiwcVImvRAYI9-XeGlqCOl8y-eYVEHSlCqThfi4' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 1501 # スロットの1台目
wait = 0.01

user_id = "g84909252+002@gmail.com"
user_pw = "sitega21"

Template_p_path = "site777/template_p.png"
Template_s_path = "site777/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]
#pmdc = "025889"

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2000":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2000":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['08/08', '025889', '1', '2051.0', '-1805.0', '0', '0']
['08/08', '025889', '2', '2564.0', '-2106.0', '0', '0']
['08/08', '025889', '3', '1026.0', '-1504.0', '0', '0']
['08/08', '025889', '5', '13333.0', '-902.0', '8', '2']
['08/08', '025889', '6', '2564.0', '-2106.0', '0', '0']
['08/08', '025889', '7', '15128.0', '41228.0', '36', '3']
['08/08', '025889', '8', '5897.0', '3913.0', '6', '2']
['08/08', '025889', '10', '1026.0', '302.0', '1', '1']
['08/08', '025889', '11', '4103.0', '-3309.0', '0', '0']
['08/08', '025888', '12', '12821.0', '-3309.0', '5', '2']
['08/08', '025888', '13', '5897.0', '-902.0', '3', '2']
['08/08', '025888', '15', '11282.0', '-2106.0', '5', '2']
['08/08', '025888', '16', '4872.0', '-4212.0', '0', '0']
['08/08', '025888', '17', '12821.0', '3010.0', '9', '4']
['08/08', '025888', '18', '4359.0', '-3911.0', '0', '0']
['08/08', '025888', '20', '7179.0', '11135.0', '11', '2']
['08/08', '025888', '21', '16154.0', '4816.0', '12', '3']
['08/08', '025888', '22', '8974.0', 

['08/08', '025895', '218', '53333.0', '-24374.0', '18', '5']
['08/08', '025895', '220', '43590.0', '-3008.0', '26', '6']
['08/08', '025895', '221', '46410.0', '7524.0', '38', '13']
['08/08', '025895', '222', '53333.0', '8728.0', '37', '6']
['08/08', '025895', '223', '51026.0', '3010.0', '35', '9']
['08/08', '025895', '225', '48718.0', '5719.0', '36', '10']
['08/08', '025895', '226', '40000.0', '18658.0', '40', '9']
['08/08', '025895', '227', '51538.0', '-1504.0', '29', '5']
['08/08', '025895', '228', '53590.0', '-9930.0', '28', '7']
['08/08', '025895', '230', '40256.0', '-29791.0', '5', '3']
['08/08', '025895', '231', '50000.0', '-11133.0', '22', '4']
['08/08', '025895', '232', '56154.0', '-9629.0', '31', '11']
['08/08', '025895', '233', '45385.0', '4214.0', '32', '10']
['08/08', '025895', '235', '49231.0', '39723.0', '58', '10']
['08/08', '025895', '236', '52308.0', '15348.0', '44', '9']
['08/08', '025895', '237', '50769.0', '-6619.0', '27', '8']
['08/08', '025895', '238', '42564.0', 

['08/08', '025606', '551', '2308.0', '-1805.0', '0', '0']
['08/08', '025606', '552', '4103.0', '-2406.0', '2', '2']
['08/08', '025606', '553', '1538.0', '-1504.0', '0', '0']
['08/08', '025606', '555', '0', '0', '0', '0']
['08/08', '025606', '556', '0', '0', '0', '0']
['08/08', '025606', '557', '1538.0', '-1203.0', '0', '0']
['08/08', '025606', '558', '7692.0', '-6018.0', '0', '0']
['08/08', '025606', '560', '0', '0', '0', '0']
['08/08', '025606', '561', '5128.0', '302.0', '3', '1']
['08/08', '025606', '562', '0', '0', '0', '0']
['08/08', '025606', '563', '1538.0', '-902.0', '1', '1']
['08/08', '025606', '565', '9487.0', '-7221.0', '1', '1']
['08/08', '025867', '566', '8205.0', '-3911.0', '2', '2']
['08/08', '025867', '567', '3590.0', '-3008.0', '0', '0']
['08/08', '025867', '568', '10769.0', '-2406.0', '4', '1']
['08/08', '025867', '570', '1795.0', '-902.0', '1', '1']
['08/08', '025867', '571', '0', '0', '0', '0']
['08/08', '025867', '572', '3846.0', '3010.0', '4', '1']
['08/08', '0258

['08/08', '025849', '781', '3846.0', '-3610.0', '0', '0']
['08/08', '025849', '782', '0', '0', '0', '0']
['08/08', '025849', '783', '0', '0', '0', '0']
['08/08', '025849', '785', '0', '0', '0', '0']
['08/08', '025849', '786', '0', '0', '0', '0']
['08/08', '025849', '787', '8462.0', '302.0', '5', '1']
['08/08', '025849', '788', '0', '0', '0', '0']
['08/08', '025864', '800', '769.0', '-902.0', '0', '0']
['08/08', '025864', '801', '0', '0', '0', '0']
['08/08', '025864', '802', '0', '0', '0', '0']
['08/08', '025864', '803', '769.0', '-902.0', '0', '0']
['08/08', '025864', '805', '769.0', '-902.0', '0', '0']
['08/08', '025864', '806', '513.0', '-902.0', '0', '0']
['08/08', '025750', '807', '7692.0', '10533.0', '30', '1']
['08/08', '025750', '808', '769.0', '-1203.0', '0', '0']
['08/08', '025750', '810', '1282.0', '-1203.0', '0', '0']
['08/08', '025750', '811', '0', '0', '0', '0']
['08/08', '025750', '812', '7179.0', '-5717.0', '2', '1']
['08/08', '025750', '813', '13077.0', '6621.0', '36', 

['08/08', '025680', '1117', '23590.0', '1806.0', '15', '2']
['08/08', '025680', '1118', '15897.0', '1205.0', '9', '2']
['08/08', '025680', '1120', '12564.0', '6621.0', '13', '1']
['08/08', '025680', '1121', '24359.0', '13242.0', '24', '1']
['08/08', '025707', '1122', '0', '0', '0', '0']
['08/08', '025707', '1123', '6923.0', '11737.0', '14', '3']
['08/08', '025707', '1125', '19744.0', '-3911.0', '11', '4']
['08/08', '025707', '1126', '14359.0', '-2406.0', '10', '4']
['08/08', '025707', '1127', '18462.0', '-12638.0', '4', '3']
['08/08', '025687', '1128', '27436.0', '7223.0', '24', '7']
['08/08', '025687', '1130', '15641.0', '27686.0', '33', '3']
['08/08', '025687', '1131', '29487.0', '-1203.0', '17', '9']
['08/08', '025825', '1132', '10000.0', '12640.0', '14', '2']
['08/08', '025825', '1133', '8974.0', '-7823.0', '0', '0']
['08/08', '025825', '1135', '12308.0', '-9629.0', '1', '1']
['08/08', '025825', '1136', '18205.0', '-12939.0', '2', '1']
['08/08', '025764', '1137', '16154.0', '3612.0

...

['08/08', '120127', '1658', '95', '-93.0', '0', '0', '--']
['08/08', '120127', '1660', '1572', '1204.0', '0', '0', '3']
['08/08', '120127', '1661', '1224', '231.0', '0', '0', '2']
['08/08', '120127', '1662', '40', '0', '0', '0', '--']
['08/08', '120127', '1663', '84', '0', '0', '0', '--']
['08/08', '120127', '1665', '2596', '278.0', '0', '0', '4']
['08/08', '120127', '1666', '33', '0', '0', '0', '--']
['08/08', '120127', '1667', '1097', '-1204.0', '0', '0', '1']
['08/08', '120127', '1668', '245', '278.0', '0', '0', '2']
['08/08', '120127', '1670', '22', '0', '0', '0', '--']
['08/08', '120127', '1671', '139', '139.0', '0', '0', '1']
['08/08', '120127', '1672', '430', '880.0', '0', '0', '3']
['08/08', '120127', '1673', '24', '0', '0', '0', '--']
['08/08', '120127', '1675', '230', '-278.0', '0', '0', '--']
['08/08', '120073', '1676', '2196', '-1250.0', '0', '7', '11']
['08/08', '120073', '1677', '4225', '-3889.0', '0', '16', '11']
['08/08', '120073', '1678', '3021', '-1667.0', '0', '9', '

['08/08', '120025', '1867', '4301', '370.0', '19', '4', '--']
['08/08', '120025', '1868', '1522', '-602.0', '5', '0', '--']
['08/08', '120025', '1870', '1973', '-602.0', '6', '3', '--']
['08/08', '120025', '1871', '1627', '-93.0', '6', '3', '--']
['08/08', '120025', '1872', '1381', '-278.0', '5', '1', '--']
['08/08', '120025', '1873', '6049', '741.0', '24', '17', '--']
['08/08', '120025', '1875', '2138', '-648.0', '3', '7', '--']
['08/08', '120025', '1876', '2460', '-93.0', '10', '4', '--']
['08/08', '120025', '1877', '2772', '-185.0', '8', '8', '--']
['08/08', '120025', '1878', '1136', '-370.0', '3', '1', '--']
['08/08', '120025', '1880', '2934', '-463.0', '9', '5', '--']
['08/08', '120025', '1881', '3291', '-93.0', '11', '8', '--']
['08/08', '120025', '1882', '8101', '1343.0', '33', '23', '--']
['08/08', '120025', '1883', '3684', '-741.0', '10', '9', '--']
['08/08', '120025', '1885', '9037', '3056.0', '45', '20', '--']
['08/08', '120025', '1886', '6854', '1852.0', '31', '19', '--']
[

['08/08', '120110', '2172', '2474', '-2037.0', '8', '1', '--']
['08/08', '120110', '2173', '2440', '-1019.0', '12', '5', '--']
['08/08', '120110', '2175', '1612', '278.0', '11', '6', '--']
['08/08', '120110', '2176', '2122', '-93.0', '13', '6', '--']
['08/08', '120110', '2177', '791', '-278.0', '3', '4', '--']
['08/08', '120110', '2178', '1502', '-93.0', '8', '7', '--']
['08/08', '120110', '2180', '220', '694.0', '4', '2', '--']
['08/08', '120110', '2181', '248', '3657.0', '16', '8', '--']
['08/08', '120110', '2182', '3610', '-2037.0', '9', '12', '--']
['08/08', '120110', '2183', '202', '-278.0', '0', '0', '--']
['08/08', '120110', '2185', '1211', '1574.0', '13', '9', '--']
['08/08', '120110', '2186', '191', '694.0', '4', '2', '--']
['08/08', '120110', '2187', '1839', '-1528.0', '4', '5', '--']
['08/08', '120110', '2188', '1685', '-1019.0', '4', '8', '--']
['08/08', '120110', '2200', '1204', '741.0', '11', '2', '--']
['08/08', '120110', '2201', '154', '-231.0', '0', '0', '--']
['08/08'

['08/08', '120122', '2380', '1741', '1435.0', '34', '2', '--']
['08/08', '120122', '2381', '1380', '926.0', '24', '2', '--']
['08/08', '120122', '2382', '2751', '-93.0', '33', '3', '--']
['08/08', '120122', '2383', '1915', '-509.0', '16', '5', '--']
['08/08', '120122', '2385', '3577', '-1806.0', '23', '8', '--']
['08/08', '120122', '2386', '1682', '-139.0', '20', '3', '--']
['08/08', '120122', '2387', '3629', '-972.0', '35', '6', '--']
['08/08', '120122', '2388', '4948', '-2037.0', '37', '10', '--']
['08/08', '120122', '2500', '3765', '1019.0', '55', '7', '--']
['08/08', '120122', '2501', '2765', '-2037.0', '12', '3', '--']
['08/08', '120122', '2502', '2786', '-1250.0', '22', '7', '--']
['08/08', '120122', '2503', '1248', '1667.0', '28', '2', '--']
['08/08', '120122', '2505', '5800', '463.0', '63', '17', '--']
['08/08', '120122', '2506', '2391', '-833.0', '23', '3', '--']
['08/08', '120122', '2507', '4442', '509.0', '45', '15', '--']
['08/08', '120122', '2508', '4070', '-417.0', '42', 

{'spreadsheetId': '1B75CIiwcVImvRAYI9-XeGlqCOl8y-eYVEHSlCqThfi4',
 'updates': {'spreadsheetId': '1B75CIiwcVImvRAYI9-XeGlqCOl8y-eYVEHSlCqThfi4',
  'updatedRange': 'S20RAW!F91929:M92505',
  'updatedRows': 577,
  'updatedColumns': 8,
  'updatedCells': 4616}}

In [3]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]
#pmdc = "025889"

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2000":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2000":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['07/19', '025889', '1', '35641.0', '3310.0', '22', '9']
['07/19', '025889', '2', '3077.0', '-903.0', '1', '1']
['07/19', '025889', '3', '9231.0', '-7523.0', '0', '0']
['07/19', '025889', '5', '5385.0', '-3009.0', '1', '1']
['07/19', '025889', '6', '7692.0', '-4514.0', '1', '1']
['07/19', '025889', '7', '6154.0', '-3611.0', '1', '1']
['07/19', '025889', '8', '10769.0', '-7523.0', '1', '1']
['07/19', '025889', '10', '11282.0', '-7824.0', '1', '1']
['07/19', '025889', '11', '3846.0', '3310.0', '4', '1']
['07/19', '025889', '12', '8974.0', '1806.0', '6', '1']
['07/19', '025889', '13', '7949.0', '3611.0', '7', '1']
['07/19', '025889', '15', '14103.0', '-10231.0', '1', '1']
['07/19', '025889', '16', '14359.0', '7523.0', '13', '4']
['07/19', '025889', '17', '1795.0', '-1204.0', '0', '0']
['07/19', '025889', '18', '6410.0', '-5417.0', '0', '0']
['07/19', '025889', '20', '23846.0', '36111.0', '37', '5']
['07/19', '025889', '21', '6923.0', '-1204.0', '3', '1']
['07/19', '025889', '22', '20256.0

['07/19', '025856', '216', '19744.0', '-903.0', '11', '5']
['07/19', '025856', '217', '30000.0', '-5417.0', '13', '5']
['07/19', '025856', '218', '5897.0', '-4815.0', '0', '0']
['07/19', '025856', '220', '20256.0', '-5417.0', '8', '3']
['07/19', '025856', '221', '13077.0', '-3611.0', '5', '4']
['07/19', '025856', '222', '17692.0', '-10833.0', '3', '1']
['07/19', '025856', '223', '23333.0', '-903.0', '13', '5']
['07/19', '025856', '225', '22821.0', '-1806.0', '12', '1']
['07/19', '025856', '226', '18718.0', '1806.0', '12', '3']
['07/19', '025856', '227', '28718.0', '-18657.0', '4', '3']
['07/19', '025886', '228', '20256.0', '3009.0', '14', '4']
['07/19', '025886', '230', '13846.0', '-2106.0', '7', '2']
['07/19', '025886', '231', '16154.0', '34306.0', '32', '3']
['07/19', '025886', '232', '17692.0', '8426.0', '16', '2']
['07/19', '025886', '233', '16923.0', '-12639.0', '2', '2']
['07/19', '025886', '235', '17692.0', '12639.0', '20', '5']
['07/19', '025886', '236', '19744.0', '-15949.0', 

['07/19', '025606', '532', '4103.0', '6620.0', '7', '1']
['07/19', '025606', '533', '3846.0', '8426.0', '8', '1']
['07/19', '025606', '535', '3333.0', '12338.0', '10', '1']
['07/19', '025606', '536', '7949.0', '-6319.0', '0', '0']
['07/19', '025606', '537', '2564.0', '-1806.0', '0', '0']
['07/19', '025606', '538', '1026.0', '-1204.0', '0', '0']
['07/19', '025606', '550', '769.0', '3009.0', '3', '1']
['07/19', '025606', '551', '1538.0', '-903.0', '1', '1']
['07/19', '025606', '552', '3077.0', '-2106.0', '0', '0']
['07/19', '025606', '553', '2821.0', '-1806.0', '0', '0']
['07/19', '025606', '555', '1026.0', '-903.0', '0', '0']
['07/19', '025606', '556', '0', '0', '0', '0']
['07/19', '025606', '557', '3846.0', '-3009.0', '0', '0']
['07/19', '025606', '558', '3590.0', '-602.0', '3', '1']
['07/19', '025606', '560', '0', '0', '0', '0']
['07/19', '025606', '561', '3333.0', '602.0', '2', '1']
['07/19', '025606', '562', '513.0', '-602.0', '0', '0']
['07/19', '025606', '563', '5385.0', '12940.0'

['07/19', '025779', '762', '6667.0', '-5116.0', '1', '1']
['07/19', '025779', '763', '4615.0', '-3611.0', '0', '0']
['07/19', '025849', '765', '2051.0', '-602.0', '1', '1']
['07/19', '025849', '766', '3333.0', '-2708.0', '0', '0']
['07/19', '025849', '767', '1282.0', '-903.0', '0', '0']
['07/19', '025849', '768', '3590.0', '4514.0', '5', '1']
['07/19', '025849', '770', '0', '0', '1', '1']
['07/19', '025849', '771', '1795.0', '-1505.0', '0', '0']
['07/19', '025849', '772', '0', '0', '0', '0']
['07/19', '025849', '773', '1795.0', '-1204.0', '0', '0']
['07/19', '025849', '775', '1282.0', '-1204.0', '0', '0']
['07/19', '025849', '776', '7949.0', '-6921.0', '0', '0']
['07/19', '025864', '777', '5128.0', '1505.0', '8', '1']
['07/19', '025864', '778', '0', '0', '0', '0']
['07/19', '025864', '780', '0', '0', '0', '0']
['07/19', '025864', '781', '0', '0', '0', '0']
['07/19', '025864', '782', '1538.0', '-1204.0', '0', '0']
['07/19', '025864', '783', '0', '0', '0', '0']
['07/19', '025864', '785',

['07/19', '025543', '1083', '15385.0', '4815.0', '17', '2']
['07/19', '025642', '1085', '0', '0', '0', '0']
['07/19', '025794', '1086', '2308.0', '-1505.0', '0', '0']
['07/19', '025668', '1087', '22564.0', '1806.0', '20', '3']
['07/19', '025782', '1088', '12051.0', '-7222.0', '6', '4']
['07/19', '025656', '1100', '7692.0', '2407.0', '7', '2']
['07/19', '025567', '1101', '7949.0', '6319.0', '17', '2']
['07/19', '025726', '1102', '1026.0', '-903.0', '2', '1']
['07/19', '025737', '1103', '6667.0', '903.0', '13', '1']
['07/19', '025734', '1105', '3077.0', '-2407.0', '0', '0']
['07/19', '025724', '1106', '2821.0', '-1204.0', '3', '1']
['07/19', '025838', '1107', '32564.0', '-4213.0', '17', '6']
['07/19', '025838', '1108', '11282.0', '3009.0', '9', '3']
['07/19', '025838', '1110', '23590.0', '17454.0', '28', '6']
['07/19', '025838', '1111', '14359.0', '-9630.0', '2', '1']
['07/19', '025779', '1112', '24359.0', '22269.0', '43', '5']
['07/19', '025779', '1113', '17692.0', '13241.0', '31', '6']

...

['07/19', '120073', '1683', '6427', '-463.0', '0', '21', '41']
['07/19', '120073', '1685', '8165', '93.0', '0', '22', '55']
['07/19', '120073', '1686', '3440', '-2176.0', '0', '5', '14']
['07/19', '120073', '1687', '7012', '-463.0', '0', '23', '45']
['07/19', '120073', '1688', '3151', '-2269.0', '0', '9', '10']
['07/19', '120073', '1700', '7511', '-648.0', '0', '22', '50']
['07/19', '120073', '1701', '8581', '5648.0', '0', '25', '79']
['07/19', '120073', '1702', '7687', '-139.0', '0', '24', '50']
['07/19', '120077', '1703', '4845', '-1389.0', '0', '16', '36']
['07/19', '120077', '1705', '7815', '-324.0', '0', '27', '55']
['07/19', '120077', '1706', '6173', '-1528.0', '0', '16', '47']
['07/19', '120149', '1707', '8563', '3796.0', '0', '24', '132']
['07/19', '120149', '1708', '8181', '2870.0', '0', '19', '114']
['07/19', '120126', '1710', '9490', '556.0', '0', '22', '44']
['07/19', '120126', '1711', '8259', '-2639.0', '0', '22', '36']
['07/19', '120126', '1712', '7724', '3889.0', '0', '2

['07/19', '120025', '1888', '2648', '-1620.0', '4', '4', '--']
['07/19', '120025', '2000', '1395', '-602.0', '4', '0', '--']
['07/19', '120025', '2001', '6475', '-463.0', '24', '12', '--']
['07/19', '120025', '2002', '2964', '-972.0', '8', '5', '--']
['07/19', '120025', '2003', '7710', '-833.0', '21', '23', '--']
['07/19', '120025', '2005', '1413', '-694.0', '2', '2', '--']
['07/19', '120025', '2006', '3770', '-556.0', '13', '4', '--']
['07/19', '120025', '2007', '3024', '-741.0', '9', '6', '--']
['07/19', '120025', '2008', '8675', '4028.0', '41', '36', '--']
['07/19', '120025', '2010', '8458', '1991.0', '37', '25', '--']
['07/19', '120025', '2011', '2199', '-1111.0', '4', '4', '--']
['07/19', '120025', '2012', '7932', '1528.0', '34', '22', '--']
['07/19', '120025', '2013', '3729', '-602.0', '11', '6', '--']
['07/19', '120025', '2015', '3621', '-185.0', '14', '3', '--']
['07/19', '120025', '2016', '5967', '-185.0', '21', '14', '--']
['07/19', '120025', '2017', '8728', '602.0', '33', '2

['07/19', '120110', '2202', '1334', '463.0', '10', '5', '--']
['07/19', '120110', '2203', '1787', '463.0', '14', '4', '--']
['07/19', '120110', '2205', '1564', '93.0', '9', '6', '--']
['07/19', '120110', '2206', '2127', '3472.0', '27', '12', '--']
['07/19', '120110', '2207', '1597', '139.0', '8', '9', '--']
['07/19', '120110', '2208', '3097', '-324.0', '17', '10', '--']
['07/19', '120110', '2210', '3090', '-463.0', '19', '7', '--']
['07/19', '120110', '2211', '1668', '1574.0', '17', '7', '--']
['07/19', '120110', '2212', '879', '1481.0', '12', '4', '--']
['07/19', '120110', '2213', '3567', '-833.0', '18', '10', '--']
['07/19', '120110', '2215', '1387', '602.0', '10', '6', '--']
['07/19', '120110', '2216', '3932', '1852.0', '35', '13', '--']
['07/19', '120110', '2217', '876', '-417.0', '2', '4', '--']
['07/19', '120110', '2218', '3542', '880.0', '25', '12', '--']
['07/19', '120110', '2220', '6594', '-3565.0', '24', '20', '--']
['07/19', '120110', '2221', '2272', '-1944.0', '5', '3', '--

['07/19', '120104', '2516', '1574', '-926.0', '0', '1', '7']
['07/19', '120142', '2517', '7268', '-4815.0', '0', '13', '39']
['07/19', '120142', '2518', '7967', '-1111.0', '0', '13', '57']
['07/19', '120142', '2520', '8568', '3657.0', '0', '24', '71']
['07/19', '120129', '2521', '7471', '-2454.0', '0', '25', '36']
['07/19', '120129', '2522', '5685', '-694.0', '0', '18', '32']
['07/19', '120129', '2523', '8169', '-2222.0', '0', '32', '42']
['07/19', '120129', '2525', '7181', '-1019.0', '0', '24', '45']
['07/19', '120129', '2526', '4380', '-3148.0', '0', '18', '18']
['07/19', '120129', '2527', '6716', '-1667.0', '0', '25', '40']
['07/19', '120129', '2528', '5877', '-741.0', '0', '15', '35']
['07/19', '120122', '2530', '6424', '-2269.0', '49', '20', '--']
['07/19', '120122', '2531', '6457', '-3611.0', '41', '12', '--']
['07/19', '120122', '2532', '6325', '1852.0', '81', '15', '--']
['07/19', '120122', '2533', '6535', '-3565.0', '45', '13', '--']
['07/19', '120122', '2535', '5363', '-694.0

{'spreadsheetId': '1B75CIiwcVImvRAYI9-XeGlqCOl8y-eYVEHSlCqThfi4',
 'updates': {'spreadsheetId': '1B75CIiwcVImvRAYI9-XeGlqCOl8y-eYVEHSlCqThfi4',
  'updatedRange': 'S20RAW!F81009:M81538',
  'updatedRows': 530,
  'updatedColumns': 8,
  'updatedCells': 4240}}